Notebook to look at T/E for 60 keV gammas, and compare superpulses from the 60 keV sideband region with 60 keV peak region after T/E cuts

Also to look at some features in the T/E vs E plot and show they are just from noise

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
import matplotlib as mpl
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

import lmfit
# from lmfit.models import SkewedGaussianModel, SkewedVoigtModel
from lmfit.lineshapes import gaussian2d, lorentzian

from astropy.modeling import models, fitting

import scipy.stats as stats
import scipy.optimize as opt
from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
import cage_utils
plt.rcParams['figure.figsize'] = [14, 10]

Load data

In [ ]:
run = 60 #44 #70 #64
dsp_id = '02'
user = True
hit =True
cal = True
lowE=False
etype = 'trapEftp_cal'

corr_DCR=True
corr_AoE=True
corr_ToE=True

norm = True
cut = True


dsp_list = ['energy', 'trapEftp', 'trapEmax', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']

df_raw, dg, runtype, rt_min, radius, angle_det, rotary = cage_utils.getDataFrame(run, user=user, hit=hit, cal=cal, dsp_list=dsp_list, lowE=lowE)

df = cage_utils.apply_DC_Cuts(run, df_raw)

# df_60 = df.query('trapEftp_cal > 55 and trapEftp_cal < 65').copy()
# df_60 = df.query('trapEftp_cal > 55 and trapEftp_cal < 80').copy()


Now open json file with cut parameters and load them

In [ ]:
with open('./analysis_60keV.json') as f:
        params = json.load(f)

In [ ]:
mean_60 = params[str(run)]['mean_60']
sig_60 = params[str(run)]['sig_60']
chiSquare_fit_60 = params[str(run)]['chiSquare_fit_60']
cut_60_3sig = params[str(run)]['cut_60_3sig']
bkg_60_left = params[str(run)]['bkg_60_left']
bkg_60_right = params[str(run)]['bkg_60_right']
bkg_60 = params[str(run)]['bkg_60']

In [ ]:

cut_60_3sig = f'({mean_60-3*sig_60} <= trapEftp_cal <= {mean_60+3*sig_60})'
cut_60_left = f'({mean_60-3*sig_60} <= trapEftp_cal < {mean_60})'
cut_60_right = f'({mean_60} <= trapEftp_cal < {mean_60+3*sig_60})'


bkg_60_left_min = mean_60-7.*sig_60
bkg_60_left_max = mean_60-4*sig_60

bkg_60_right_min = mean_60+4*sig_60
bkg_60_right_max = mean_60+7.*sig_60

bkg_60_left = f'({mean_60-7.*sig_60} <= trapEftp_cal < {mean_60-4*sig_60})'
bkg_60_right = f'({mean_60+4*sig_60} < trapEftp_cal <= {mean_60+7.*sig_60})'


counts_peak = len(df.query(cut_60_3sig).copy())
err_peak = np.sqrt(counts_peak)

left_counts = len(df.query(bkg_60_left))
right_counts = len(df.query(bkg_60_right))

total_bkg = len(df.query(bkg_60).copy())
err_bkg = np.sqrt(total_bkg)


counts = counts_peak - total_bkg
err = np.sqrt(counts_peak + total_bkg)
print(f'bkg_subtracted counts: {counts}')
print(f'error: {err}')
print(f'{(err/counts)*100:.3f}%')


Create new dataframes with energy cuts for the sideband and 60 keV peak regions

In [ ]:
df_60 = df.query(cut_60_3sig).copy()
df_bkg = df.query(bkg_60).copy()

Plots T/E vs Energy

In [ ]:
fig, ax = plt.subplots()
elo, ehi, epb = 50, 70, 0.5
ToElo, ToEhi, ToEpb= 0.3,  0.5, 0.0025

nbx = int((ehi-elo)/epb)
nby = int((ToEhi-ToElo)/ToEpb)



# fig.suptitle(f'T/E vs Energy', horizontalalignment='center', fontsize=16)

ToE_hist_norm, xedges, yedges = np.histogram2d(df['trapEftp_cal'], df['ToE'], bins=[nbx, nby], range=([elo, ehi], [ToElo, ToEhi]))
X, Y = np.mgrid[elo:ehi:nbx*1j, ToElo:ToEhi:nby*1j]


pcm = plt.pcolormesh(X, Y, ToE_hist_norm,  shading='nearest') #0.002, 0.2

cb = plt.colorbar()
cb.set_label("counts", ha = 'right', va='center', rotation=270, fontsize=14)
cb.ax.tick_params(labelsize=12)
ax.set_xlabel(f'Energy (keV)', fontsize=16)
ax.set_ylabel('T/E (arb)', fontsize=16)
plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)


ax.text(0.95, 0.83, f'r = {radius} mm \ntheta = {angle_det} deg', verticalalignment='bottom',
            horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

# plt.legend()
plt.title(f'T/E vs Energy \n{runtype} run {run}, {rt_min:.2f} mins', fontsize=18)
plt.tight_layout()
# plt.ylim(-0.05, 0.1)
# plt.ylim(-0.05, 0.08)
# plt.ylim(0.42, 0.51)
# plt.xlim(52.5, 65)
plt.show()

1D T/E

In [ ]:
ToElo, ToEhi, ToEpb= 0.1,  0.5, 0.0025
nbx = int((ToEhi-ToElo)/ToEpb)

# bl_sig_lo, bl_sig_hi = 3.1, 5. # run 70
# bl_sig_lo, bl_sig_hi = 3.1, 5.25
# bl_sig_lo_raw, bl_sig_hi_raw = quant_raw_sig[0], quant_raw_sig[1]  

ToE_hist, bins = np.histogram(df['ToE'], bins=nbx,
                range=[ToElo, ToEhi])

plt.plot(bins[1:], ToE_hist, ds='steps', c='b', lw=1)


# plt.axvline(bl_sig_lo_raw, c='r', lw=1)
# plt.axvline(bl_sig_hi_raw, c='r', lw=1)

plt.xlabel('ToE', fontsize=16)
plt.ylabel('counts', fontsize=16)
plt.show()

In [ ]:
ToElo, ToEhi, ToEpb= 0.3,  0.5, 0.0025
nbx = int((ToEhi-ToElo)/ToEpb)

print(nbx)

# bl_sig_lo, bl_sig_hi = 3.1, 5. # run 70
# bl_sig_lo, bl_sig_hi = 3.1, 5.25
# bl_sig_lo_raw, bl_sig_hi_raw = quant_raw_sig[0], quant_raw_sig[1]  

ToE_hist, bins = np.histogram(df_60['ToE'], bins=nbx,
                range=[ToElo, ToEhi])

plt.plot(bins[1:], ToE_hist, ds='steps', c='b', lw=1)


# plt.axvline(bl_sig_lo_raw, c='r', lw=1)
# plt.axvline(bl_sig_hi_raw, c='r', lw=1)

plt.xlabel('ToE', fontsize=16)
plt.ylabel('counts', fontsize=16)

# plt.xlim(0.35, 0.5)
plt.show()

Come up with T/E cut based on fit of the 1D T/E distribution. 
Start with 60 keV peak region

As is, works only for earlier runs

In [ ]:
# First use gauss_mode_width_max to use for initial guesses in fit_hist

if run < 66:
    ToE_60_hist, bins, tvars = pgh.get_hist(df_60['ToE'], bins=50, range=[0.43, 0.5]) #range=[54, 65]
    # plt.plot(bins[1:], ToE_60_hist, ds='steps', c='b', lw=1)
    pars, cov = pgf.gauss_mode_width_max(ToE_60_hist, bins, tvars)
    mode = pars[0]
    width = pars[1]
    amp = pars[2]
    print(f'mode: {mode}')
    print(f'width: {width}')
    print(f'amp: {amp}')

In [ ]:
if run < 66: 
    Tpars, Tcov = pgf.fit_hist(cage_utils.gauss_fit_func, ToE_60_hist, bins, tvars, guess = (amp, mode, width, 1))

    chi_2_ToE_60 = pgf.goodness_of_fit(ToE_60_hist, bins, cage_utils.gauss_fit_func, Tpars)


    ToE_60_mean = Tpars[1]
    ToE_60_sig = Tpars[2]
    ToE_60_amp = Tpars[0]
    ToE_60_const = Tpars[3]


    print(f'chi square: {chi_2_ToE_60}')
    print(f'mean: {ToE_60_mean}')
    print(f'width: {ToE_60_sig}')
    print(f'amp: {ToE_60_amp}')
    print(f'C: {ToE_60_const}')
    
    plt.plot(bins[1:], cage_utils.gauss_fit_func(bins[1:], *Tpars), c = 'r')
    plt.plot(bins[1:], ToE_60_hist, ds='steps', c='b', lw=1)

Repeat fit for sideband region

In [ ]:
# First use gauss_mode_width_max to use for initial guesses in fit_hist

ToE_60_main, bins, tvars = pgh.get_hist(df_60['ToE'], bins=100, range=[0.37, 0.42]) #range=[0.35, 0.42] for run < 66
# plt.plot(bins[1:], ToE_60_main, ds='steps', c='b', lw=1)
pars, cov = pgf.gauss_mode_width_max(ToE_60_main, bins, tvars)
mode = pars[0]
width = pars[1]
amp = pars[2]
print(f'mode: {mode}')
print(f'width: {width}')
print(f'amp: {amp}')

In [ ]:
plt.plot(bins[1:], ToE_60_main, ds='steps', c='b', lw=1)

In [ ]:
main_Tpars, main_Tcov = pgf.fit_hist(cage_utils.gauss_fit_func, ToE_60_main, bins, tvars, guess = (amp, mode, width, 1))

chi_2_ToE_main = pgf.goodness_of_fit(ToE_60_main, bins, cage_utils.gauss_fit_func, main_Tpars)

ToE_main_mean = main_Tpars[1]
ToE_main_sig = main_Tpars[2]
ToE_main_amp = main_Tpars[0]
ToE_main_const = main_Tpars[3]

# fwhm = width_fit*2.355

print(f'chi square: {chi_2_ToE_main}')
print(f'mean: {ToE_main_mean}')
print(f'width: {ToE_main_sig}')
print(f'amp: {ToE_main_amp}')
print(f'C: {ToE_main_const }')


In [ ]:
plt.plot(bins[1:], cage_utils.gauss_fit_func(bins[1:], *main_Tpars), c = 'r')
plt.plot(bins[1:], ToE_60_main, ds='steps', c='b', lw=1)

Define T/E cuts for 60 keV and sideband regions

In [ ]:
ToE_60_hi = 2*ToE_60_sig + ToE_60_mean
ToE_60_lo = ToE_60_mean - 2*ToE_60_sig 

ToE_bkg_hi = (2*ToE_main_sig + ToE_main_mean)
ToE_bkg_lo = (ToE_main_mean - 2*ToE_main_sig)

if ToE_60_lo < ToE_bkg_hi:
    ToE_60_lo = ToE_bkg_hi 

print(f'ToE background: {ToE_bkg_lo:.3f}, {ToE_bkg_hi:.3f}')
print(f'ToE 60 keV: {ToE_60_lo:.3f}, {ToE_60_hi:.3f}')

In [ ]:
ToE_60 = f'({ToE_60_lo} <= ToE < {ToE_60_hi})'
ToE_left = f'({ToE_60_lo} <= ToE < {ToE_60_mean})'
ToE_right = f'({ToE_60_mean} <= ToE < {ToE_60_hi})'

ToE_bkg = f'({ToE_bkg_lo} < ToE < {ToE_bkg_hi})'

# ToE_bkg_left = f'({ToE_bkg_lo}<= ToE <{ToE_main_mean})'
# ToE_bkg_right = f'({ToE_main_mean}<= ToE <{ToE_bkg_hi})'

bkg_left = f'({ToE_bkg} and {bkg_60_left})' 
bkg_right = f'({ToE_bkg} and {bkg_60_right})' 
bkg = f'{bkg_left} or {bkg_right}'

ToE_60_3sigma = f'{cut_60_3sig} and {ToE_60}'

ToE_60_left = f'{cut_60_left} and {ToE_right}'
ToE_60_right = f'{cut_60_right} and {ToE_left}'


# print(ToE_left)
# print(ToE_60_left)
# print(ToE_60_right)

print(ToE_60_3sigma)
print(bkg)

# print(bkg_left)
# print(bkg_right)
# print(bkg)

Now look at superpulses. 

In [ ]:
times, all_60_raw = cage_utils.get_superpulse_taligned(df, dg, ToE_60_3sigma, all=True, norm=False)

In [ ]:
times, bkg_60_raw = cage_utils.get_superpulse_taligned(df, dg, bkg, all=True, norm=False)

In [ ]:
fig, ax = plt.subplots()
plt.plot(times[:-1], all_60_raw[:-1], '-b', label=('60 keV'))
plt.plot(times[:-1], bkg_60_raw[:-1], '-g', label=('background'))


# plt.xlim(0, 4300)
# plt.ylim(-0.025, 0.1)

plt.xlim(3700, 4300)
# plt.ylim(-0.025, 0.2)

# plt.xlim(3700, 3710)
# plt.ylim(-0.025, 0.1)


# plt.xlim(3800, 8000)
# plt.ylim(0.75, 1)

# plt.xlim(3800, 4300)
# plt.ylim(0.95, 1)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

plt.xlabel('samples', fontsize = 16)
plt.ylabel('normalized', fontsize=16)

plt.legend(fontsize=16)

plt.title(f'run {run}; {radius} mm', fontsize = 16)
plt.show()

Now, notch filter superpulses to remove digitizer noise

In [ ]:
#pre_bkg_60_notched = notchFilter(bkg_60, f_notch=25e6, Q=20)
bkg_60_notched = cage_utils.notchFilter_SIS3302(bkg_60_raw, Q=20)

#pre_all_60_notched = notchFilter(all_60, f_notch=25e6, Q=20)
all_60_notched = cage_utils.notchFilter_SIS3302(all_60_raw, Q=20)

bkg_60 = np.divide(bkg_60_notched, np.amax(bkg_60_notched))
all_60 = np.divide(all_60_notched, np.amax(all_60_notched))

In [ ]:
fig, ax = plt.subplots()
plt.plot(times[:-1], all_60[:-1], '-b', label=('60 keV'))
plt.plot(times[:-1], bkg_60[:-1], '-g', label=('background'))


# plt.xlim(0, 4300)
# plt.ylim(-0.025, 0.1)

# plt.xlim(3700, 4300)
# plt.ylim(-0.025, 0.2)

# plt.xlim(3700, 3710)
# plt.ylim(-0.025, 0.1)


# plt.xlim(3800, 8000)
# plt.ylim(0.75, 1)

plt.xlim(3800, 4300)
plt.ylim(0.95, 1.001)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

plt.xlabel('samples', fontsize = 16)
plt.ylabel('normalized', fontsize=16)

plt.legend(fontsize=16)

plt.title(f'run {run}; {radius} mm', fontsize = 16)
plt.show()

Save the results of the T/E cuts, if necessary

In [ ]:
nwfs_60 = len(df.query(ToE_60_3sigma).copy())
nwfs_bkg = len(df.query(bkg).copy())

print(nwfs_60, nwfs_bkg)

In [ ]:
#Cuts to write:
# write_params = True
write_params = False

param_keys = ['ToE_60', 'ToE_bkg', 'bkg', 'ToE_60_3sigma']
param_list = [ToE_60, ToE_bkg, bkg, ToE_60_3sigma]


if write_params == True:
    for key, cut in zip(param_keys, param_list):
        cage_utils.writeJson('./analysis_60keV.json', run, key, cut)


PZ-correct notched superpulses

In [ ]:

all_60_pz = cage_utils.double_pole_zero([all_60_notched], 21250, 433, 0.045)[0]

Etrap_all_60 = cage_utils.trap_norm(all_60_pz, 100, 400)
trap0_all_60 = cage_utils.asymTrapFilter(all_60_pz, 100, 1, 400)
max_trap0_all_60 = np.argmax(trap0_all_60)

t0_all_60 = cage_utils.time_point_thresh_max(trap0_all_60, 0.0, max_trap0_all_60, 
                                            max_trap0_all_60 - 200)

Eftp_all_60 = Etrap_all_60[t0_all_60 + 400]

all_60_pz_norm = np.divide(all_60_pz, Eftp_all_60)

# print(np.amax(all_60_pz_norm))

In [ ]:
Save superpulses

In [ ]:
# waveform_dict = {'run': , 'times':np.array(times), 'pure_60_wf': np.array(all_60), 'bkg_60': np.array(bkg_60), 'left_60': np.array(left_60), 'right_60': np.array(right_60)}
# bkg_and_60 = np.zeros(len(all_60))

wf_dict = {'run': run, 'nwfs_bkg': nwfs_bkg, 'nwfs_60': nwfs_60, 'samples': [times], 'bkg': [bkg_60], 'pure_60': [all_60], 
          'bkg_raw': [bkg_60_raw], 'pure_60_raw': [all_60_raw], 'pure_60_pz': [all_60_pz_norm]}
wf_df = pd.DataFrame(data=wf_dict)
print(wf_df)

In [ ]:
save_superpulse = False

if save_superpulse ==True:
    outfile = f'./data/normScan/run{run}_superpulses.hdf5'
    wf_df.to_hdf(outfile, key='superpulses', mode='w')

Load superpulses obtained from the sideband subtration method and compare them to superpulses from T/E cut
(Did this to validate sideband subtraction method)

In [ ]:
f_superpulse = './data/normScan/superpulses_60keV_allRuns.hdf5'
# f_superpulse = './data/normScan/superpulses_1sig_60keV_allRuns.hdf5'
data_superpulse = pd.read_hdf(f_superpulse, key = '/superpulses')

data_superpulse = data_superpulse.query(f'run=={run}').copy()
    
bkg_sub_wf = np.array(data_superpulse['pure_60_pz'][0])

In [ ]:
print(bkg_sub_wf)

Make plots of superpulses comparing T/E cut to the superpulses from sideband subtraction

In [ ]:
fig, ax = plt.subplots()
plt.plot(times, all_60_pz_norm, '-b', lw=2, label=('T/E cut'))
plt.plot(times, bkg_sub_wf, '-g', lw=2, label=('sideband \nsubtraction'))

#bl
# plt.xlim(0, 4300)
# plt.ylim(-0.025, 0.1)

#rise
plt.xlim(3700, 3875)
plt.ylim(-0.023, 0.2)

#rise tail
# plt.xlim(3750, 3875)
# plt.ylim(-0.025, 1.01)

#tail
# plt.xlim(3800, 4300)
# plt.ylim(0.955, 1.01)

#fullTail
# plt.xlim(3800, 8000)
# plt.ylim(0.75, 1.01)

plt.setp(ax.get_xticklabels(), fontsize=28) #16
plt.setp(ax.get_yticklabels(), fontsize=28)
plt.xlabel('clock cycles', fontsize=32) #20
plt.ylabel('normalized ADU', fontsize=32)

# plt.setp(ax.get_xticklabels(), fontsize=14)
# plt.setp(ax.get_yticklabels(), fontsize=14)

# plt.xlabel('samples', fontsize = 16)
# plt.ylabel('normalized', fontsize=16)

plt.legend(fontsize=28)

plt.title(f'T/E cut vs. sideband subtraction \nrun {run}; {radius} mm', fontsize = 32)
# plt.savefig(f'./plots/new_normScan/60keV_analysis/1sig/waveforms/run{run}_wf_bkg_sub.png', dpi=200)
plt.savefig(f'./plots/new_normScan/60keV_analysis/waveforms/diss_run{run}_wf_bkg_sub_rise.png', dpi=200)
plt.savefig(f'./plots/new_normScan/60keV_analysis/waveforms/diss_run{run}_wf_bkg_sub_rise.pdf', dpi=200)
plt.show()

In [ ]:
diff = bkg_sub_wf - all_60_pz_norm

In [ ]:
fig, ax = plt.subplots()
plt.plot(diff)

plt.ticklabel_format(axis="y", style="sci", scilimits=(-0.1,0.1), useMathText=True)
ax.yaxis.get_offset_text().set_fontsize(24)



plt.setp(ax.get_xticklabels(), fontsize=28) #16
plt.setp(ax.get_yticklabels(), fontsize=28)
plt.xlabel('clock cycles', fontsize=32) #20
plt.ylabel('normalized ADU', va='bottom', fontsize=32)

# plt.legend(fontsize=16)

plt.title(f'Difference \nrun {run}; {radius} mm', fontsize = 32)
# plt.savefig(f'./plots/new_normScan/60keV_analysis/1sig/waveforms/run{run}_wf_bkg_sub_diff.png', dpi=200)
plt.savefig(f'./plots/new_normScan/60keV_analysis/waveforms/run{run}_wf_bkg_sub_diff.png', dpi=200)
plt.savefig(f'./plots/new_normScan/60keV_analysis/waveforms/run{run}_wf_bkg_sub_diff.pdf', dpi=200)

Now look at superpulses from the upper left blob on the T/E vs Energy hist and compare them to 
superpulses from the bottom right. This was to show the non-flat shape was likely just do to noise

In [ ]:
times, left_60 = cage_utils.get_superpulse_taligned(df, dg, ToE_60_left, all=True, norm=False)

In [ ]:
times, right_60 = cage_utils.get_superpulse_taligned(df, dg, ToE_60_right, all=True, norm=False)

In [ ]:
#pre_bkg_60_notched = notchFilter(bkg_60, f_notch=25e6, Q=20)
left_60_notched = cage_utils.notchFilter_SIS3302(left_60, Q=20)

#pre_all_60_notched = notchFilter(all_60, f_notch=25e6, Q=20)
right_60_notched = cage_utils.notchFilter_SIS3302(right_60, Q=20)


In [ ]:
fig, ax = plt.subplots()
plt.plot(times, left_60_notched, '-b', label=('left'))
plt.plot(times, right_60_notched, '-g', label=('right'))
plt.legend()

# plt.xlim(3900, 4100)

# plt.xlim(0, 4300)
# plt.ylim(-2, 15)

# plt.xlim(3700, 8000)
# plt.ylim(120, 175)

plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)

plt.xlabel('samples', fontsize = 16)
plt.ylabel('ADC', fontsize=16)

plt.legend(fontsize=16)

plt.title(f'run {run}; {radius} mm', fontsize = 16)

plt.savefig(f'./plots/new_normScan/60keV_analysis/waveforms/run{run}_ToE_noiseComp.png', dpi=200)

plt.show()

Tried an additional basline subtraction 

In [ ]:
bl_mean_left = np.mean(left_60_notched[:3800])
bl_mean_right = np.mean(right_60_notched[:3800])

bl_sub_left = left_60_notched - bl_mean_left
bl_sub_right = right_60_notched - bl_mean_right

In [ ]:
fig, ax = plt.subplots()
plt.plot(times, bl_sub_left[:len(bl_sub_left)-1], '-b', label=('left'))
plt.plot(times, bl_sub_right[:len(bl_sub_right)-1], '-g', label=('right'))
plt.legend()

# plt.xlim(3900, 4100)

# plt.xlim(0, 4300)
# plt.ylim(-2, 15)

# plt.xlim(3800, 8000)
# plt.ylim(120, 175)

# plt.xlim(3900, 4200)
# plt.ylim(25, 140)

plt.xlim(3500, 6500)

plt.setp(ax.get_xticklabels(), fontsize=14)
plt.setp(ax.get_yticklabels(), fontsize=14)

plt.xlabel('samples', fontsize = 16)
plt.ylabel('ADC', fontsize=16)

plt.legend(fontsize=16)

plt.title(f'run {run}; {radius} mm', fontsize = 16)

plt.show()